In [1]:
import pandas as pd
import plotly.graph_objects as go
from collections import defaultdict
from s2sphere import CellId, LatLng, Cell

In [2]:
# http://s2geometry.io/about/overview
# http://s2geometry.io/devguide/s2cell_hierarchy.html
def adaptive_partition(df: pd.DataFrame, t1: int = 10_000, t2: int = 50, max_level: int = 18):
    all_cells = defaultdict(list)
    
    # Loop through all lat,lon values
    for _, row in df.iterrows():
        lat, lon = row['lat'], row['lon']
        latlng = LatLng.from_degrees(lat, lon)
        cell_id = CellId.from_lat_lng(latlng)
        
        # Add the lat,lon to be cells at each hierarchy
        for level in range(max_level + 1):
            parent = cell_id.parent(level)
            all_cells[parent.id()].append((lat, lon))
            
    final_cells = set()
    processed = set()
    
    def recurse(cell_id_val: CellId, depth: int):
        """Recursive function to calculate and slice cells of the grid"""
        # Cell already processed
        if cell_id_val in processed:
            return
        processed.add(cell_id_val)
        
        points_list = all_cells.get(cell_id_val, [])
        # No points exist in the cell
        if len(points_list) == 0:
            return
            
        # Not big enough to split or done splitting
        if len(points_list) <= t1 or depth >= max_level:
            # Only keep cells that contain enough points
            if len(points_list) >= t2:
                final_cells.add(cell_id_val)
            return
        
        # Too many points, recurse through each split of the cell
        cell_id_obj = CellId(cell_id_val)
        for i in range(4):
            child = cell_id_obj.child(i)
            recurse(child.id(), depth + 1)
    
    # Iterate through each level and recurve through the cells
    for level in range(max_level + 1):
        level_cells = [cid for cid in all_cells.keys() if CellId(cid).level() == level]
        for cell_id_val in level_cells:
            if cell_id_val not in processed:
                recurse(cell_id_val, level)
    
    return final_cells

def get_cell_vertices(cell_id_val: CellId):
    """Grabs all 4 vertices of a cell"""
    cell_id_obj = CellId(cell_id_val)
    cell = Cell(cell_id_obj)
    vertices = []
    for i in range(4):
        vertex = cell.get_vertex(i)
        latlng = LatLng.from_point(vertex)
        vertices.append((latlng.lat().degrees, latlng.lng().degrees))
    return vertices

In [3]:
sampled_df = pd.read_csv('../data/imgs/sampled.csv', index_col=0)
if 'lat' in sampled_df.columns and 'lon' in sampled_df.columns:
    sampled_df = sampled_df.drop(columns=['lat', 'lon'])
points_df = pd.read_csv('../data/points.csv')
points_df = points_df.rename(columns={'id': 'orig_id'})
points_df = points_df[['orig_id', 'lat', 'lon']]
city_df = pd.merge(sampled_df, points_df, on=['orig_id'])
print("Data shape:", city_df.shape)
display(city_df.head())

Data shape: (190693, 8)


,uuid,source,orig_id,city,country,iso3,lat,lon
0,99891cd8-bd32-4cc9-9e48-9d7fbdc30909,Mapillary,755195782374799,Washington,United States,USA,38.892015,-77.031205
1,0752026e-3449-4e9a-a75b-974e16738eca,Mapillary,3983548141753049,Washington,United States,USA,38.903943,-77.018739
2,7e2600a1-eaaf-4d91-8d4d-065984e90cde,Mapillary,233657401861198,Washington,United States,USA,38.899477,-77.031631
3,2e92aa72-215d-4672-aa1f-bcd71b02322d,Mapillary,958812788229569,Washington,United States,USA,38.902491,-77.029299
4,e9e94c21-5d7b-4f4d-9fb0-42c71745b376,Mapillary,287795916343455,Washington,United States,USA,38.893817,-77.015253


In [4]:
# t1=5_000, t2=50, max_level=30 for 1,000,000 looks pretty good
# Level 0: 6 cells
# Level 1: 24 cells
# ...
# Total cells = 6 * 4^level
max_level = 25
final_cells = adaptive_partition(city_df, t1=10000, t2=1000, max_level=max_level)

In [5]:
def latlon_to_cellid(lat: float, lon: float):
    """Find which final cell this lat/lon belongs to"""
    latlng = LatLng.from_degrees(lat, lon)
    cell_id = CellId.from_lat_lng(latlng)
    
    # Traverse from max_level down to find the corresponding cell
    # Each lat,lon only belongs to the highest level cell
    for level in range(max_level, -1, -1):
        parent = cell_id.parent(level)
        parent_id = parent.id()
        
        if parent_id in final_cells:
            return parent_id
    
    return None

# Add the cell id corresponding to each latitude and longitude
city_df['s2_cell_id'] = city_df.apply(
    lambda row: latlon_to_cellid(row['lat'], row['lon']), 
    axis=1
)

# Remove null cell_ids
city_df.dropna(subset=['s2_cell_id'], inplace=True)
cell_ids = [int(id) for id in set(city_df['s2_cell_id'].unique())]
print(f"Number of unique cells: {len(cell_ids)}")

Number of unique cells: 144


In [6]:
def visualize_s2_cells(final_cells: set[CellId]):
    fig = go.Figure()
    
    for cell_id_val in final_cells:
        vertices = get_cell_vertices(cell_id_val)
        lats = [v[0] for v in vertices] + [vertices[0][0]]
        lons = [v[1] for v in vertices] + [vertices[0][1]]
        
        fig.add_trace(go.Scattergeo(
            lon=lons,
            lat=lats,
            mode='lines',
            line=dict(width=1, color='blue'),
            showlegend=False
        ))
    
    fig.update_layout(
        title=f'Adaptive S2 Cell Partitioning - {len(final_cells)} Cells',
        geo=dict(
            projection_type='natural earth',
            showland=True,
            showcountries=True,
        ),
        height=720,
        width=1280,
    )
    return fig

fig = visualize_s2_cells(cell_ids)
fig.show()

In [7]:
# Create the labels as a sequence from 0 to num_unique_cells
replacement_dict = dict()
for i, cell_id in enumerate(cell_ids):
    replacement_dict[cell_id] = i
    
city_df['label'] = city_df['s2_cell_id'].replace(replacement_dict).astype(int)
city_df.to_csv('../data/imgs/sampled.csv')
print(f"Number of unique cells: {len(city_df['s2_cell_id'].unique())}")
print(city_df['label'].unique())

Number of unique cells: 144
[  9  10   7   8   6  11  12  16  17  18  50  51  52  53  54  55  56  57
  58  59  74 126  70  38  85 140 138  86  13 131  24  25  94 120  36 137
  37  15 124 100  77 130  84 107  97   3 111  14  41 142 108  23  19  20
   4   5  42  43  44  80  45 109 112  46  47 114  21  48  40 110 113 115
 121 106  22  87  49  81 122 103 141  67  32 105  60 123  93  33  68 127
 134 135  26  82  79 128 101  27  61 118 116  78   0  34  75  88  98  35
  31  71  83 104  62 132  28  89  63  39  29 102  90  64 139 117 129  65
 136  72  99  95 125 133   1  73   2  69 119  91  96  30  92 143  76  66]


In [10]:
# Saves class labels back to the sampled csv
city_df.to_csv('../data/imgs/sampled.csv')